In [22]:
import pandas as pd
import numpy as np

In [23]:
#read the data
data = pd.read_csv('phone usage data.csv',encoding='utf-8')

In [24]:
'''------------------------------- Scaling Starts -----------------------------------------------'''
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

numerical_cols = ['Age']

data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

'''------------------------------- Scaling Ends -----------------------------------------------'''


'------------------------------- Scaling Ends -----------------------------------------------'

In [25]:
'''--------------------------------------------Encoding Starts--------------------------------------------'''
binary_cols = [
    'Do you check your screen time stats regularly?',
    'Have you ever used any app to try to reduce your screen time?',
    'Do you usually scroll on your phone before going to bed?',
    'Do you currently have a job? (Part-time or Full-time)'
]
for col in binary_cols:
    data[col] = data[col].map({'Yes': 1, 'No': 0})


hours_mapping = {'1-3 hours': 1, '4-6 hours': 2, '7-9 hours': 3, '10+ hours': 4}
data['On average, how many hours a day do you spend on your phone?'] = data[
    'On average, how many hours a day do you spend on your phone?'].map(hours_mapping)


short_video_mapping = {
    "I don't watch short-video content": 0,
    'A little of my phone usage': 1,
    'Some of my phone usage': 2,
    'Most of my phone usage': 3
}
data['How much short-video content (TikTok, Reels, Shorts) do you watch?'] = data[
    'How much short-video content (TikTok, Reels, Shorts) do you watch?'].map(short_video_mapping)

focus_mapping = {
    'Less than 10 minutes': 0,
    '10-20 minutes': 1,
    '20-40 minutes': 2,
    'Over 40 minutes': 3,
    "I don't usually feel that urge": 4
}
data['How long can you focus on a single task (like studying or reading) before feeling the urge to check your phone?'] = data[
    'How long can you focus on a single task (like studying or reading) before feeling the urge to check your phone?'].map(focus_mapping)


reread_mapping = {'No': 0, 'Rarely': 1, 'Sometimes': 2, 'Yes, often': 3}
data['Have you ever re-read the same paragraph or page multiple times because your mind wandered?'] = data[
    'Have you ever re-read the same paragraph or page multiple times because your mind wandered?'].map(reread_mapping)


gpa_mapping = {
    'Less than 2.0 (Poor)': 0,
    '2.0-2.5 (Satisfactory)': 1,
    '2.5-3.0 (Good)': 2,
    '3.0-3.5 (Very good)': 3,
    '3.5 or Higher (Excellent)': 4
}
data['What is your GPA or grade (Excellent, Very good, Good, etc.)?'] = data[
    'What is your GPA or grade (Excellent, Very good, Good, etc.)?'].map(gpa_mapping)



data = pd.get_dummies(data, columns=['Gender'], prefix='Gender')


app_categories = [
    'Social Media (Instagram, Facebook, TikTok, etc.)', 'Texting', 'Streaming and Videos (YouTube, Netflix, Shahid, etc.)',
    'Study or Work related (Canvas, Notion, LinkedIn, etc.)', 'Gaming', 'Shopping (Amazon, Noon, Jumia, etc.)'
]
for app in app_categories:
    data[f'App_{app}'] = data['What apps do you spend the most time on? (Pick no more than 3)'].apply(
        lambda x: 1 if app in str(x) else 0)


data.drop('What apps do you spend the most time on? (Pick no more than 3)', axis=1, inplace=True)


data.to_csv('preprocessed_phone_usage_data.csv', index=False)

'''--------------------------------------------Encoding Ends--------------------------------------------'''


'--------------------------------------------Encoding Ends--------------------------------------------'

In [26]:
data.head()

,Age,"On average, how many hours a day do you spend on your phone?",Do you check your screen time stats regularly?,How satisfied are you with your phone usage?,Have you ever used any app to try to reduce your screen time?,How often do you check you phone first thing when you wake up?,Do you usually scroll on your phone before going to bed?,Do you think your phone hurts your academic performance?,Have you ever caught yourself scrolling endlessly with no purpose?,"How much short-video content (TikTok, Reels, Shorts) do you watch?",...,"What is your GPA or grade (Excellent, Very good, Good, etc.)?",Do you currently have a job? (Part-time or Full-time),Gender_Female,Gender_Male,"App_Social Media (Instagram, Facebook, TikTok, etc.)",App_Texting,"App_Streaming and Videos (YouTube, Netflix, Shahid, etc.)","App_Study or Work related (Canvas, Notion, LinkedIn, etc.)",App_Gaming,"App_Shopping (Amazon, Noon, Jumia, etc.)"
0,0.1875,3,1,2,1,5,1,4,5,2,...,4.0,0.0,False,True,1,1,1,0,0,0
1,0.1250,3,1,1,1,5,1,3,5,3,...,4.0,0.0,False,True,1,1,1,1,0,0
2,0.1875,4,0,1,1,4,1,5,5,3,...,1.0,0.0,False,True,1,1,0,0,1,0
3,0.2500,2,1,4,1,3,1,4,3,2,...,2.0,1.0,False,True,1,1,1,0,0,0
4,0.2500,2,1,2,0,5,1,4,4,2,...,4.0,0.0,True,False,1,0,1,0,0,0


In [27]:
data.shape

(88, 22)

In [28]:
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata

# Load data
df = pd.read_csv("preprocessed_phone_usage_data.csv")

# Auto-detect metadata
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(df)

# Initialize with metadata
model = CTGANSynthesizer(metadata)
model.fit(df)

synthetic_data = model.sample(1000)
synthetic_data.to_csv("synthetic_phone_usage.csv", index=False)

C:\Users\Computer Store\venv\Lib\site-packages\sdv\single_table\base.py:126: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
C:\Users\Computer Store\venv\Lib\site-packages\sdv\single_table\base.py:105: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [29]:
synthetic_data.shape

(1000, 22)